In [32]:
import os
import json
import numpy as np
import trimesh

from planner_schema import validate_plan, PLAN_DEFAULT
from llm_planner import llm_make_plan_local, get_llm

In [33]:
from phase1_scene_graph_final import (
    aabb_from_mesh_world,
    robust_floor_z,
    tag_objects_phase15,
    build_named_targets,
    build_occupancy_grid,
    generate_candidates,
    score_candidates_weighted,
    object_margin,
    plot_topdown
)

In [34]:
def build_placement_facts(best, plan, user_prompt, target_name):
    """Convert solver output → grounded measurable facts"""

    def safe(v):
        try:
            return float(v)
        except:
            return 0.0

    return {
        "user_prompt": user_prompt,
        "chosen_point_xyz": best.get("point"),
        "score": safe(best.get("score")),
        "components": best.get("components", {}),
        "distances_m": {
            "clearance": safe(best.get("d_clear")),
            "to_wall": safe(best.get("d_wall")),
            "to_target": safe(best.get("d_target")),
        },
        "plan": {
            "object_type": plan.get("object_type"),
            "dimensions": plan.get("object_dims"),
            "target": target_name,
            "weights": plan.get("weights"),
            "wall_preference": plan.get("wall_pref"),
        }
    }

In [35]:
def generate_grounded_explanation(user_prompt, best, plan, target_name):
    """LLM explains placement using real numbers only"""

    facts = build_placement_facts(best, plan, user_prompt, target_name)

    prompt = f"""
You are explaining a spatial AI placement decision.

Rules:
- Use ONLY numbers from the facts
- Do NOT invent measurements
- 4-6 sentences

FACTS:
{json.dumps(facts, indent=2)}

Explain why this placement satisfies the user request.
"""

    llm = get_llm()
    out = llm(prompt, max_tokens=220, temperature=0.3)

    explanation = out["choices"][0]["text"].strip()
    return explanation, facts

In [36]:
def run_pipeline(glb_path: str, user_prompt: str):
    print("\n========== SPATIAL AI PIPELINE ==========")

    # ---------------- Load Scene ----------------
    scene = trimesh.load(glb_path, force="scene")
    if not isinstance(scene, trimesh.Scene):
        raise ValueError("Input must be a GLB scene")

    objects = []
    all_verts = []

    for geom_name, mesh in scene.geometry.items():
        nodes = scene.graph.geometry_nodes.get(geom_name, [])

        for node_name in nodes:
            transform, _ = scene.graph.get(node_name)

            v = mesh.vertices
            if len(v) > 20000:
                v = v[np.random.choice(len(v), 20000, replace=False)]

            v_h = np.hstack([v, np.ones((len(v), 1))])
            v_world = (transform @ v_h.T).T[:, :3]
            all_verts.append(v_world)

            obj = {"node_name": node_name, "geometry_name": geom_name}
            obj.update(aabb_from_mesh_world(mesh, transform))
            objects.append(obj)

    if not all_verts:
        raise RuntimeError("Scene contains no geometry")

    all_verts = np.vstack(all_verts)
    room_min = all_verts.min(axis=0)
    room_max = all_verts.max(axis=0)
    floor_z = robust_floor_z(all_verts)

    # ---------------- Tag Objects ----------------
    objects = tag_objects_phase15(objects, room_min, room_max, floor_z)
    obstacles = [o for o in objects if o.get("tag") == "obstacle"]
    targets = build_named_targets(obstacles)

    print("Detected targets:", list(targets.keys()))

    # ---------------- LLM PLAN ----------------
    context = {
        "room_size_m": {
            "width": float(room_max[0]-room_min[0]),
            "depth": float(room_max[1]-room_min[1]),
            "height": float(room_max[2]-floor_z)
        },
        "num_obstacles": len(obstacles),
        "available_targets": list(targets.keys()),
        "supported_objects": ["chair", "table", "sofa", "plant"]
    }

    raw_plan = llm_make_plan_local(user_prompt, context)
    plan = validate_plan(raw_plan or {}, available_targets=context["available_targets"])

    print("LLM PLAN:", plan)

    # ---------------- Candidate Generation ----------------
    w = plan["object_dims"]["width"]
    d = plan["object_dims"]["depth"]

    margin = object_margin(w, d, clearance=plan["constraints"]["min_clearance"])

    candidates, meta = generate_candidates(
        room_min, room_max, floor_z,
        obstacles,
        step=0.25,
        clearance=margin,
        boundary_margin=plan["constraints"]["boundary_margin"]
    )

    print("Valid placements:", meta["num_valid"])

    # ---------------- Scoring ----------------
    target_name = plan["target"]["name"]
    target_xy = targets.get(target_name)

    scored = score_candidates_weighted(
        candidates,
        obstacles,
        room_min,
        room_max,
        target_xy,
        weights=plan["weights"],
        wall_pref=plan["wall_pref"]
    )

    best = scored[0]
    print("Best placement:", best["point"], "score:", best["score"])

    # ---------------- Explanation ----------------
    explanation, facts = generate_grounded_explanation(user_prompt, best, plan, target_name)

    # ---------------- Save Outputs ----------------
    os.makedirs("outputs", exist_ok=True)

    json.dump(plan, open("outputs/plan.json","w"), indent=2)
    json.dump(best, open("outputs/best.json","w"), indent=2)
    json.dump(facts, open("outputs/facts.json","w"), indent=2)
    open("outputs/explanation.txt","w").write(explanation)

    print("\n===== EXPLANATION =====")
    print(explanation)

    return best, plan, explanation

In [37]:
if __name__ == "__main__":
    run_pipeline(
        "living_room.glb",
        "Place a chair near the wall but keep walking space"
    )


========== SPATIAL AI PIPELINE ==========
Detected targets: ['window', 'tv', 'shelf']


Llama.generate: 167 prefix-match hit, remaining 8 prompt tokens to eval
llama_perf_context_print:        load time =   12325.89 ms
llama_perf_context_print: prompt eval time =     621.64 ms /     8 tokens (   77.70 ms per token,    12.87 tokens per second)
llama_perf_context_print:        eval time =   24316.46 ms /   136 runs   (  178.80 ms per token,     5.59 tokens per second)
llama_perf_context_print:       total time =   25072.81 ms /   144 tokens
llama_perf_context_print:    graphs reused =        131
Llama.generate: 1 prefix-match hit, remaining 466 prompt tokens to eval


LLM PLAN: {'object_type': 'chair', 'object_dims': {'width': 0.6, 'depth': 0.6}, 'target': {'name': None}, 'weights': {'near_target': 0.4210526315789474, 'max_clearance': 0.10526315789473685, 'near_wall': 0.4736842105263158}, 'constraints': {'min_clearance': 0.3, 'boundary_margin': 0.1}, 'wall_pref': 'near'}
Valid placements: 130
Best placement: [7.027899910272897, -0.5975192089530289, -1.734517626987245] score: 0.5789473684210527


llama_perf_context_print:        load time =   12325.89 ms
llama_perf_context_print: prompt eval time =   21993.64 ms /   466 tokens (   47.20 ms per token,    21.19 tokens per second)
llama_perf_context_print:        eval time =   39422.57 ms /   219 runs   (  180.01 ms per token,     5.56 tokens per second)
llama_perf_context_print:       total time =   61674.95 ms /   685 tokens
llama_perf_context_print:    graphs reused =        211



===== EXPLANATION =====
This placement is suitable for a chair as it is located 0.09999999999999998 meters from the wall, ensuring the chair is near the wall while maintaining walking space. The clearance distance of 3.9730876892422757 meters from any obstacle is more than the required maximum clearance of 1.0 meter. The chair's dimensions, with a width and depth of 0.6 meters each, will not encroach upon the walking space. The 'near_wall' component, which prioritizes placing the chair near the wall, has a score of 1.0, indicating a strong preference for this placement. The overall score of 0.5789473684210527 reflects the balance between the 'near_target', 'max_clearance', and 'near_wall' components, making this placement an optimal solution for the user request.
